<a href="https://colab.research.google.com/github/narfdf999/Arquitectura/blob/master/PMV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Primera aproximación al problema de generar edificio a lo largo de un camino dibujado por el arquitecto***

## Librerías

---



In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.layers import Dense, Dropout, Input
from keras.models import Model,Sequential
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam

Using TensorFlow backend.


## Visualización y preprocesamiento de datos

---



### Prueba con fichero 17_a

In [8]:
direcciones_v=pd.read_csv('17_a_direcciones.csv',sep=',')
print(direcciones_v)
print(direcciones_v.shape)
direcciones=np.genfromtxt('17_a_direcciones.csv',delimiter=',')
print(direcciones)
print(direcciones.shape)
puntos_v=pd.read_csv('17_a_puntos.csv',sep=',')
print(puntos_v)
print(puntos_v.shape)
puntos=np.genfromtxt('17_a_puntos.csv',delimiter=',')
print(puntos)
print(puntos.shape)

      -0.923764078523375  ...  -0.94776374744804.59
0              -0.923764  ...             -0.947764
1              -0.923764  ...             -0.947764
2              -0.923764  ...             -0.947764
3              -0.923764  ...             -0.947764
4              -0.923764  ...             -0.947764
...                  ...  ...                   ...
1013           -0.937844  ...             -0.934182
1014           -0.937844  ...             -0.934182
1015           -0.937844  ...             -0.934182
1016           -0.937844  ...             -0.934182
1017           -0.937844  ...             -0.934182

[1018 rows x 1259 columns]
(1018, 1259)
[[-0.92376408 -0.91654703 -0.90399378 ... -0.9401907  -0.94578498
  -0.94776375]
 [-0.92376408 -0.91654703 -0.90399378 ... -0.9401907  -0.94578498
  -0.94776375]
 [-0.92376408 -0.91654703 -0.90399378 ... -0.9401907  -0.94578498
  -0.94776375]
 ...
 [-0.93784381 -0.93195966 -0.92172482 ... -0.92463978 -0.93168862
  -0.93418188]
 [-0.9

### Matriz de ruido aleatorio de forma (1019,128)

In [10]:
v_aleatorio=np.random.random((1019,128))
print(v_aleatorio)
print(v_aleatorio.shape)

[[0.3311625  0.37076839 0.11976353 ... 0.59068364 0.68798263 0.55211693]
 [0.61709054 0.95573608 0.53073195 ... 0.66753843 0.97285412 0.44090502]
 [0.2564563  0.54718938 0.88359918 ... 0.94994209 0.39674961 0.97691121]
 ...
 [0.02886195 0.06331291 0.47179617 ... 0.37323079 0.02448928 0.12389139]
 [0.24763466 0.22670585 0.69965164 ... 0.74398553 0.50455257 0.19077683]
 [0.47962823 0.4108557  0.15178256 ... 0.76361444 0.1268029  0.43931936]]
(1019, 128)


### Concatenación de ruido con puntos para la entrada de la GAN

In [14]:
gen_input=np.concatenate([v_aleatorio,puntos], axis=1)
print(gen_input)
print(gen_input.shape)

[[ 3.31162498e-01  3.70768394e-01  1.19763530e-01 ...  9.21602370e-01
  -5.85746679e-01  0.00000000e+00]
 [ 6.17090545e-01  9.55736082e-01  5.30731952e-01 ...  9.21836604e-01
  -5.78241624e-01  9.81354269e-04]
 [ 2.56456300e-01  5.47189379e-01  8.83599177e-01 ...  9.22070869e-01
  -5.70735582e-01  1.96270854e-03]
 ...
 [ 2.88619524e-02  6.33129097e-02  4.71796173e-01 ...  9.45310318e-01
   1.73875582e-01  9.97055937e-01]
 [ 2.47634665e-01  2.26705850e-01  6.99651636e-01 ...  9.45463455e-01
   1.78782228e-01  9.98037291e-01]
 [ 4.79628226e-01  4.10855701e-01  1.51782558e-01 ...  9.45615168e-01
   1.83643243e-01  9.99018646e-01]]
(1019, 132)


Optimizador ADAM


In [0]:
def adam_optimizer():
    return Adam(lr=0.0002, beta_1=0.5)

## Implementación de la GAN

---



Generador

In [15]:
def create_generator():
  generator=Sequential()
  generator.add(Dense(units=256,input_dim=132))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(units=512))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(units=1024))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(units=1259, activation='tanh'))
  #poner la salida a 1259 

  generator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
  return generator
  
g=create_generator()
g.summary()
  








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               34048     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 1024)              0         
_____________________

Discriminador

In [16]:
def create_discriminator():
    discriminator=Sequential()
    discriminator.add(Dense(units=1024,input_dim=1263))
    discriminator.add(LeakyReLU(0.2))
       
    discriminator.add(Dense(units=512))
    discriminator.add(LeakyReLU(0.2))
       
    discriminator.add(Dense(units=256))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Dense(units=128))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Dense(units=64))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Dense(units=32))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(units=1, activation='softmax'))
    
    discriminator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    return discriminator
d =create_discriminator()
d.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1024)              1294336   
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)              

GAN

In [17]:
def create_gan(discriminator, generator):
  discriminator.trainable=False
  gan_input = Input(shape=(132,))
  x = generator(gan_input)
  p = Input(shape=(4,))
  x = keras.layers.Concatenate()([x,p])
  gan_output=discriminator(x)
  gan = Model(inputs=[gan_input, p], output=gan_output)
  gan.compile(loss='binary_crossentropy', optimizer='adam')
  return gan

gan=create_gan(d,g)
gan.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 132)          0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 1259)         1981419     input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 4)            0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 1263)         0           sequential_1[1][0]               
                                                                 input_2[0][0]              

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("se...)`
  


## Entrenamiento

In [0]:
def training(epochs=1, batch_size=12):
    
    #Loading the data
    #(X_train, y_train, X_test, y_test) = load_data()
    batch_count = X_train.shape[0] / batch_size
    
    # Creating GAN
    generator= create_generator()
    discriminator= create_discriminator()
    gan = create_gan(discriminator, generator)
    
    for e in range(1,epochs+1 ):
        print("Epoch %d" %e)
        for _ in tqdm(range(batch_size)):
        #generate  random noise as an input  to  initialize the  generator
            noise= np.random.normal(0,1, [batch_size, 132])
            # cambiar a 128
            
            # Generate fake MNIST images from noised input
            generated_images = generator.predict(noise)
            print(generated_images.shape)
            
            # Get a random set of  real images
            image_batch =X_train[np.random.randint(low=0,high=X_train.shape[0],size=batch_size)]
            print(image_batch.shape)
            
            #Construct different batches of  real and fake data 
            X= np.concatenate([image_batch, generated_images])
            
            # Labels for generated and real data
            y_dis=np.zeros(2*batch_size)
            y_dis[:batch_size]=0.9
            
            #Pre train discriminator on  fake and real data  before starting the gan. 
            discriminator.trainable=True
            discriminator.train_on_batch(X, y_dis)
            
            #Tricking the noised input of the Generator as real data
            noise= np.random.normal(0,1, [batch_size, 132])
            y_gen = np.ones(batch_size)
            
            # During the training of gan, 
            # the weights of discriminator should be fixed. 
            #We can enforce that by setting the trainable flag
            discriminator.trainable=False
            
            #training  the GAN by alternating the training of the Discriminator 
            #and training the chained GAN model with Discriminator’s weights freezed.
            gan.train_on_batch(noise, y_gen)
            
        if e == 1 or e % 20 == 0:
           
            plot_generated_images(e, generator)
training(400,128)

NameError: ignored

In [0]:
generator

NameError: ignored

Postprocesado de datos de salida

In [0]:
df = pd.read_csv('17_a.csv', header=None)

In [0]:
df[0]=df[0].apply(lambda x: x.split(','))

In [0]:
df[0]

0       [13.3780471856351, 2.8377602455671, 1.48631674...
1       [13.4278691159807, 2.84197631680024, 1.4863167...
2       [13.477697602257, 2.8461141852625, 1.486316745...
3       [13.5275327187439, 2.8501714213576, 1.48631674...
4       [13.5773745379712, 2.85414546918325, 1.4863167...
                              ...                        
1014    [18.3546830379293, 1.41693950216082, 3.6487514...
1015    [18.3878800275203, 1.37955020045437, 3.6487514...
1016    [18.4207616530758, 1.34188325604917, 3.6487514...
1017    [18.4533341781645, 1.30394869912087, 3.6487514...
1018    [18.4856037816336, 1.26575612534079, 3.6487514...
Name: 0, Length: 1019, dtype: object